<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/Fimetuning%20SciBERT%20with%20SpaCy%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import io
from nltk.tokenize import TreebankWordTokenizer as twt

In [ ]:
%mkdir data

In [ ]:
%cd data

/content/data


# Load Data

## Load datasets

In [ ]:
train_df = pd.read_csv('/content/cdr_dner_train_df.csv')
test_df = pd.read_csv('/content/cdr_dner_test_df.csv')
dev_df = pd.read_csv('/content/cdr_dner_dev_df.csv')


In [ ]:
iob_train = pd.read_csv('/content/iob_train.csv')
iob_test = pd.read_csv('/content/iob_test.csv')
iob_dev = pd.read_csv('/content/iob_dev.csv')

In [ ]:
iob_train['tokens'] = iob_train['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
iob_train['iob_tags'] = iob_train['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

iob_test['tokens'] = iob_test['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
iob_test['iob_tags'] = iob_test['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

iob_dev['tokens'] = iob_dev['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
iob_dev['iob_tags'] = iob_dev['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
false_ents =['\'' '2\'', '\"",', 'sensitisation',  '\'2', 'phenethylamino', '3\'', '\'flattened', '5\'', '2\'']

## Convert to tsv

In [ ]:
def to_tsv_iob(df):
  tokens, tags = [], []
  for i,j in zip(df['tokens'], df['iob_tags']):  
    for k,l in zip(i,j):
      if not k.lower().startswith(tuple(false_ents)):
        tokens.append(k)
        tags.append(l)
  iob_tags = pd.DataFrame({"tokens":tokens, "tags":tags}).set_index('tokens')
  return iob_tags

In [ ]:
to_tsv_iob(iob_train)['tags'].value_counts()

O             89528
B-Chemical     5024
B-Disease      4047
I-Disease      2209
I-Chemical      751
Name: tags, dtype: int64

In [ ]:
train_iob = to_tsv_iob(iob_train)
test_iob = to_tsv_iob(iob_test)
dev_iob = to_tsv_iob(iob_dev)

In [ ]:
%pwd

'/content/data'

In [ ]:
train_iob.to_csv('/content/data/iob_tsv_train.tsv', sep = '\t')
test_iob.to_csv('/content/data/iob_tsv_test.tsv', sep = '\t')
dev_iob.to_csv('/content/data/iob_tsv_dev.tsv', sep = '\t')

## Create .json files

In [ ]:
!python -m spacy convert /content/data/iob_tsv_train.tsv ./ -t json -s -c conll
!python -m spacy convert /content/data/iob_tsv_dev.tsv ./ -t json -s -c conll
!python -m spacy convert /content/data/iob_tsv_test.tsv ./ -t json -s -c conll


ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_train.json
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_dev.json
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_test.json


In [ ]:
# %cd data

## Create .spacy files

In [ ]:
!python -m spacy convert /content/data/iob_tsv_dev.json ./ -t spacy
!python -m spacy convert /content/data/iob_tsv_train.json ./ -t spacy
!python -m spacy convert /content/data/iob_tsv_test.json ./ -t spacy

✔ Generated output file (877 documents): iob_tsv_dev.spacy
✔ Generated output file (855 documents): iob_tsv_train.spacy
✔ Generated output file (909 documents): iob_tsv_test.spacy


# Initializing SpaCy Pipeline

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-07-01 13:24:45--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-07-01 13:24:45--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?6jRlN0-QgB4CVkP4RyftipgeDnlfKKKaSHSauX_OSUSVkMqKCDeTZEML0ZYGIqF8yVrpHmYYdtJZ9_U-eDpLcXS5rkSrPgSuRP2HRPJ6hWUKZ

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
%cd ..

/content


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 27 kB/s 
     |████████████████████████████████| 51 kB 319 kB/s 
     |████████████████████████████████| 4.2 MB 9.2 MB/s 
     |████████████████████████████████| 1.1 MB 61.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.9 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 577.3 MB 3.4 kB/s 
     |████████████████████████████████| 12.5 MB 33.1 MB/s 
     |████████████████████████████████| 7.6 MB 4.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0+cu113
    Uninstalling torchaudio-0.11.0+cu113:
      Successfully uninstalled torchaudio-0.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
!pip install -U spacy[cuda92,transformers]
!export CUDA_PATH=”/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH
!pip install cupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.0 MB 1.1 MB/s 
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.7 MB/s 
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
  ERROR: Failed cleaning build dir for cupy
Failed to build cupy
    Running setup.py install for cupy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9lg8qww7/cupy_255cc541d3dc4050a39fb0c69631ab10/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9lg8qww7/cupy_255cc541d3dc4050a39fb0c69631ab10/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exi

In [ ]:
!pip install git+https://github.com/explosion/spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-zk_gocpp
  Running command git clone -q https://github.com/explosion/spacy-transformers /tmp/pip-req-build-zk_gocpp


In [ ]:
!python -m spacy init fill-config /content/config.cfg /content/config.cfg

In [ ]:
!python -m spacy debug data /content/config.cfg

In [ ]:
!pip uninstall cupy
!pip install cupy-cuda100

# Finetune SciBERT with Spacy Pipeline

In [ ]:
!python -m spacy train -g 0 /content/config.cfg --output ./

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-07-01 12:18:22,392] [INFO] Set up nlp object from config
[2022-07-01 12:18:22,402] [INFO] Pipeline: ['transformer', 'ner']
[2022-07-01 12:18:22,406] [INFO] Created vocabulary
[2022-07-01 12:18:22,407] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 